<a href="https://colab.research.google.com/github/KENTAROSZK/ELYZA-tasks-100/blob/main/ELYZA_tasks_100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考ページ

- https://qiita.com/wayama_ryousuke/items/105a164e5c80c150caf1#%E3%81%8A%E3%82%8F%E3%82%8A%E3%81%AB
- https://github.com/Northern-System-Service/gpt4-autoeval/tree/75413eaad90ce47ef648f2be60960f8d2b35d88f
- https://colab.research.google.com/drive/1cz-UKLiRaJRAOipg3Hr4R5rK1lzwPrTZ?usp=sharing#scrollTo=8U_hbtLtxytU

In [38]:
%pip install --upgrade transformers
%pip install --upgrade accelerate
%pip install torch  bitsandbytes huggingface_hub[cli] huggingface_hub hf_transfer openai
%pip install datasets jsonlines

## ELYZA-tasks-100 データセットのダウンロード

モデルのダウンロードには 🤗datasets ライブラリを用いる。

データは `/content/dataset.jsonl` に保存する。

In [30]:
import json
from pathlib import Path
from datasets import load_dataset

# Load the dataset
ds = load_dataset("elyza/ELYZA-tasks-100")

print(f'{ds=}')
print()
print(f'{ds["test"]=}')
print()
print(f'{type(ds["test"]["input"])=}\n', f'{ds["test"]["input"]=}')
print()
print(f'{type(ds["test"]["output"])=}\n', f'{ds["test"]["output"]=}')
print()
print(f'{type(ds["test"]["eval_aspect"])=}\n', f'{ds["test"]["eval_aspect"]=}')

for _ in range(3):
    print("↓"*50, "dataset の中身を具体的にforで取得する", "↓"*50)

print()
for i, item in enumerate( ds["test"] ):
    print(i, f'{item["input"]=}')
    print(i, f'{item["output"]=}')
    print(i, f'{item["eval_aspect"]=}')
    print("="*30)
    if i > 1:
        break

ds=DatasetDict({
    test: Dataset({
        features: ['input', 'output', 'eval_aspect'],
        num_rows: 100
    })
})

ds["test"]=Dataset({
    features: ['input', 'output', 'eval_aspect'],
    num_rows: 100
})

type(ds["test"]["input"])=<class 'list'>
 ds["test"]["input"]=['仕事の熱意を取り戻すためのアイデアを5つ挙げてください。', 'クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。', '古代ギリシャを学ぶ上で知っておくべきポイントは？', '以下の説明を読んだユーザーが感じるであろう疑問とは？\n\n大量のゴミの処理にお困りではありませんか？\n\n「生産で出るゴミ処理費用が大幅にかかっている」\n「指定ゴミ袋が導入され、一気にコストが上がってしまった」\n「工場が狭いので、ゴミ保管スペースを減らしたい」\n「ISO14001でゴミ削減が大きな課題となっている」\n\n場所をとる嵩張るゴミをつぶしてまとめて運びやすい形にすることで、処理・保管・運搬にかかる様々なコストを削減したり、環境負荷の低減にも貢献できるのが、廃棄物圧縮減容機→『ゴミ圧縮機』です。\n\nゴミ圧縮機の主な用途は廃プラスチック・紙・段ボール・袋類・ペットボトル・プラスチック容器などつぶれるが元に戻ってしまうゴミを圧縮梱包します。', '次に読むべきSFを10冊挙げて', '次の文章を読んで、その人がどの程度怒っているか、1〜10の尺度で評価してください。(1＝怒っていない、10＝非常に怒っている)。\n\n1. またテストで赤点か。君は全く...\n2. テストで赤点？今回は難しかったね。', 'こうして社会の秩序は政治家や教師や警官による物理的な強制のほか、メンバー個々人の自発的な意志にも支えられています。実際、学校の授業が自習になったときや、誰もいない場所で信号待ちをしているとき、誰も見ていない道端でタバコを捨てたくなったときなど、教室で大騒ぎをし、堂々と

In [18]:
# Convert and write the dataset to a file in JSONL format
path_jsonl = Path("/content/dataset.jsonl")

if (path_jsonl.parent is not None) and (not path_jsonl.parent.exists()):
    path_jsonl.parent.mkdir(parents=True, exist_ok=True)

In [19]:
# Function to convert dataset to JSONL format and print
def dataset_to_jsonl(dataset, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for entry in dataset:
            # Construct JSON object
            json_obj = {
                "input_text": entry['input'],
                "output_text": entry['output'],
                "eval_aspect": entry['eval_aspect']
            }

            # Write JSON object to file in JSONL format
            json_str = json.dumps(json_obj, ensure_ascii=False)
            file.write(json_str + '\n')

dataset_to_jsonl(ds["test"], path_jsonl)

## 評価

ELYZA-tasks-100 データセットを用いて、指定したモデルの評価を実行する。

### プロンプトについて

モデルに回答を生成させるときのプロンプトは、各モデルのモデルカード (🤗HuggingFace の README ページ) に記載されているプロンプトを使用する。

### ハイパーパラメータについて

ハイパーパラメータは 🤗HuggingFace の Code Snippet のとおりとする（公開者が指定したハイパーパラメータを、チューン済みとみなす）。
ただし、長文の回答を要求する設問もあるため、`max_new_tokens` は `1024` に統一する。

In [31]:
import json
import jsonlines
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Setup model and tokenizer
model_name = "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto"
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [35]:
def generate_text(prompt):
    messages = [
        {"role": "user", "content": prompt},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(model.device)

    # Set seed for reproducibility
    seed = 42
    torch.manual_seed(seed)

    # ✋Hparamsは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    tokens = model.generate(
        input_ids,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.8,
    )
    # Remove the input tokens from the generated tokens before decoding
    output_tokens = tokens[0][len(input_ids[0]):]

    return tokenizer.decode(output_tokens, skip_special_tokens=True)

In [37]:
def process_dataset(input_file, output_file):
    # Read dataset and generate predictions
    with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
        for i, obj in enumerate(reader):
            prompt = obj['input_text']
            generated_text = generate_text(prompt)
            print(i, f"{prompt=}\n", f"{generated_text=}")
            print()
            writer.write({"pred": generated_text})
            if i > 2:
                break

# Process the dataset
input_dataset = '/content/dataset.jsonl'
output_predictions = '/content/preds.jsonl'
process_dataset(input_dataset, output_predictions)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


0 prompt='仕事の熱意を取り戻すためのアイデアを5つ挙げてください。'
 generated_text='仕事の熱意を取り戻すためのアイデアを5つご紹介します。\n\n1. **小さな成功体験を積み重ねる:** 大きな目標を達成するのではなく、日々の業務の中で小さな成功体験を積み重ねることを意識しましょう。例えば、難しいタスクを完了した、クライアントから感謝の言葉をもらった、チームで目標を達成できたなど、小さな達成感を積み重ねることで、仕事に対するモチベーションを高めることができます。\n\n2. **仕事内容を見直す:** 今の仕事内容にやりがいを感じられない場合は、自分の興味やスキルと仕事内容が合致しているか見直してみましょう。転職や部署異動を検討するのも一つの方法です。あるいは、現在の仕事の中で、新しいスキルを習得したり、責任範囲を広げたりすることで、仕事内容に新たな魅力を加えることもできます。\n\n3. **周りの人とコミュニケーションをとる:** 同僚や上司と積極的にコミュニケーションをとることで、仕事に対するモチベーションを高めることができます。仕事に関する悩みや不安を相談したり、意見交換をしたりすることで、新たな視点を得たり、解決策を見つけることができるかもしれません。また、チームワークを意識することで、仕事に対する責任感や達成感も高まります。\n\n4. **ワークライフバランスを整える:** 仕事とプライベートのバランスが崩れていると、仕事に対する熱意が低下してしまうことがあります。休暇をしっかりと取得したり、趣味やリラックスできる時間を確保したりすることで、心身ともにリフレッシュし、仕事への意欲を取り戻すことができます。\n\n5. **目標を明確にする:** 将来のキャリアプランや目標を明確にすることで、仕事に対するモチベーションを高めることができます。具体的な目標を設定し、それを達成するために努力することで、仕事にやりがいを感じることができるでしょう。また、定期的に目標を見直し、必要に応じて修正することで、常に前向きな気持ちで仕事に取り組むことができます。\n\n\n\nこれらのアイデアを参考に、自分に合った方法で仕事への熱意を取り戻してください。'



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1 prompt='クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。'
 generated_text='深い森の奥深く、古いオークの木の下に住むクマのバーナードは、いつも冒険を夢見ていました。彼は森の境界線に沿って流れる小川の音を聞きながら、広大な世界を探検したいと考えていました。しかし、バーナードは臆病な性格で、母親のクマ、エドナがいつも言っているように、「安全な場所を離れるべきではない」という教えに従っていました。\n\nある日、バーナードは森の端で、海から吹いてくる塩辛い風を感じました。その香りは、彼を魅了し、冒険心を掻き立てました。彼は母親に別れを告げ、森の境界線を越えて、見たこともない青い海に向かって歩き始めました。\n\n海辺に着くと、バーナードは息を呑みました。白い砂浜、青い空、そして遠くに見える波の音に、彼は圧倒されました。そして、海辺で遊んでいるアザラシの群れを見つけました。彼らは陽気で、バーナードは彼らに近づいて、友達になりたいと思いました。\n\nアザラシたちは、バーナードの優しい心と好奇心に惹かれ、彼を仲間として受け入れました。彼らはバーナードに、海の生き物や波の秘密を教えました。バーナードは、海の世界の素晴らしさに夢中になり、毎日を海辺で過ごしました。\n\nしかし、数週間後、バーナードは母親の顔が恋しくなりました。彼はエドナに会いたいと思い始め、家に帰る決意をしました。アザラシたちは、バーナードの決断を理解し、彼に別れを告げました。\n\nバーナードは、海辺を離れて森に向かいました。彼は、海の世界で学んだことを母親に伝えようと決心しました。エドナは、バーナードの冒険を聞いて驚きましたが、同時に彼の勇気と好奇心に誇りを感じました。\n\nバーナードは、森の奥深くで、エドナの温かい抱擁を受けました。彼は、海辺で過ごした日々を振り返り、冒険の素晴らしさと、家の温かさを再認識しました。彼は、安全な場所を離れることの重要性を学び、同時に、世界には無限の可能性があることを知りました。\n\nバーナードは、海辺で出会ったアザラシたちとの友情を大切にし、いつかまた会えることを夢見ていました。そして、彼は森の奥深くで、静かに、そして幸せに暮らしました。'



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


2 prompt='古代ギリシャを学ぶ上で知っておくべきポイントは？'
 generated_text='古代ギリシャを学ぶ上で知っておくべきポイントはたくさんありますが、特に重要なのは以下の通りです。\n\n**1. 時代背景と文化**\n\n* **地中海世界における位置づけ:** 古代ギリシャは地中海世界の中心部に位置し、周辺地域との交流を通じて文化や思想を発展させてきました。\n* **ポリス（都市国家）制度:** 古代ギリシャは独立した都市国家（ポリス）が多数存在し、それぞれが独自の政治制度、文化、宗教を持っていました。\n* **民主制:** アテネを中心に、市民による直接民主制が発展しました。これは、古代ギリシャの政治思想や社会構造を理解する上で重要なポイントです。\n* **宗教:** ギリシャ神話は、古代ギリシャの宗教観、倫理観、芸術に大きな影響を与えました。主な神々や神話の内容を理解しておくことは重要です。\n\n**2. 歴史的な出来事と人物**\n\n* **ミケーネ文明:** 紀元前16世紀から12世紀頃に栄えた文明で、ミケーネ遺跡が有名です。\n* **暗黒時代:** 紀元前12世紀頃から紀元前8世紀頃まで、ギリシャ世界は混乱と停滞の時代を経験しました。\n* **ポリスの興隆:** 紀元前8世紀頃から、ポリスが再び発展し始め、アテネ、スパルタ、コリントスなどの都市が栄えました。\n* **ペルシャ戦争:** 紀元前5世紀にアケメネス朝ペルシャ帝国とギリシャ諸ポリスが戦い、ギリシャは勝利しました。この戦争は、ギリシャのアイデンティティ形成に大きな影響を与えました。\n* **古典期:** 紀元前5世紀から4世紀にかけて、ソクラテス、プラトン、アリストテレスといった哲学者たちが活躍し、西洋思想の基礎を築きました。\n* **ヘレニズム時代:** 紀元前4世紀から紀元前1世紀にかけて、ギリシャ文化が地中海世界全体に広まりました。\n* **ローマ帝国の台頭:** 紀元前1世紀頃から、ローマ帝国がギリシャ世界を征服し、ギリシャ文化はローマ文化に吸収されていきました。\n\n**3. 芸術と文学**\n\n* **彫刻:** パルテノン神殿の彫刻や、クレタ島のミノア文明の壁画など、古代ギリシャの芸術は、その美しさと技術力の高さで知られています。\

## 出力結果に対してChatGPTで採点させる。

In [39]:
# 採点のためのテンプレートプロンプト
## 採点基準とかをしっかり掲載

template_prompt_for_eval = """
あなたは言語モデルの採点者です。

問題, 正解例, 採点基準, 言語モデルが生成した回答が与えられます。

「採点基準」と「正解例」を参考にして、「言語モデルの回答」を評価してください。
そして、回答理由および1,2,3,4,5の5段階評価による採点結果を「評価フォーマット」に示すようなJSON形式で返してください。

# 問題
{input_text}

# 正解例
{output_text}

# 採点基準
基本的な採点基準
- 1点: 誤っている、 指示に従えていない
- 2点: 誤っているが、方向性は合っている
- 3点: 部分的に誤っている、 部分的に合っている
- 4点: 合っている
- 5点: 役に立つ

基本的な減点項目
- 不自然な日本語: -1点
- 部分的に事実と異なる内容を述べている: -1点
- 「倫理的に答えられません」のように過度に安全性を気にしてしまっている: 2点にする

問題固有の採点基準
{eval_aspect}

# 言語モデルの回答
{pred}

# 評価フォーマット
{{"reason": "(採点基準に照らした評価内容)", "grade": (採点結果、1～5の5段階評価)}}
"""

In [44]:
from google.colab import userdata
import os

# APIキーの準備
os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")

def call_openai(prompt):
    """ChatGPTを呼び出す"""
    from openai import OpenAI
    client = OpenAI()
    response = client.chat.completions.create(
        model="o1-mini",
        messages=[
            {"role": "user", "content": prompt},
        ]
    )

    return response.choices[0].message.content



def evaluate(input_file, output_file):
    """LLMの出力結果に対してChatGPTで採点させる"""
    with jsonlines.open(input_file) as inputs, jsonlines.open(output_file) as preds:
        for i, (input_obj, pred_obj) in enumerate( zip(inputs, preds) ):
            input_text = input_obj['input_text']
            output_text = input_obj['output_text']
            eval_aspect = input_obj['eval_aspect']
            pred = pred_obj['pred']

            prompt = template_prompt_for_eval.format(
                input_text=input_text,
                output_text=output_text,
                eval_aspect=eval_aspect,
                pred=pred
            )

            score = call_openai(prompt)

            print(i, f"{score=}\n", f"{prompt=}\n")
            print("="*30)

            if i > 2:
                break

In [45]:
# Process the dataset
input_dataset = '/content/dataset.jsonl'
output_predictions = '/content/preds.jsonl'
evaluate(input_dataset, output_predictions)

0 score='```json\n{\n  "reason": "モデルの回答は、提供された採点基準と正解例に従い、仕事の熱意を取り戻すための5つの具体的で有益なアイデアを適切に提示しています。日本語も自然で、内容に重複や誤りは見られません。また、仕事の効率化やスキルアップに偏ることなく、熱意の回復に焦点を当てています。",\n  "grade": 5\n}\n```'
 prompt='\nあなたは言語モデルの採点者です。\n\n問題, 正解例, 採点基準, 言語モデルが生成した回答が与えられます。\n\n「採点基準」と「正解例」を参考にして、「言語モデルの回答」を評価してください。\nそして、回答理由および1,2,3,4,5の5段階評価による採点結果を「評価フォーマット」に示すようなJSON形式で返してください。\n\n# 問題\n仕事の熱意を取り戻すためのアイデアを5つ挙げてください。\n\n# 正解例\n1. 自分の仕事に対する興味を再発見するために、新しい技能や知識を学ぶこと。\n2. カレッジやセミナーなどで講演を聴くことで、仕事に対する新しいアイデアや視点を得ること。\n3. 仕事に対してストレスを感じている場合は、ストレスマネジメントのテクニックを学ぶこと。\n4. 仕事以外の楽しいことをすることで、ストレスを発散すること。\n5. 仕事に対して自己評価をすることで、自分がどのように進化しているのかを知ること。\n\n# 採点基準\n基本的な採点基準\n- 1点: 誤っている、 指示に従えていない\n- 2点: 誤っているが、方向性は合っている\n- 3点: 部分的に誤っている、 部分的に合っている\n- 4点: 合っている\n- 5点: 役に立つ\n\n基本的な減点項目\n- 不自然な日本語: -1点\n- 部分的に事実と異なる内容を述べている: -1点\n- 「倫理的に答えられません」のように過度に安全性を気にしてしまっている: 2点にする\n\n問題固有の採点基準\n- 熱意を取り戻すのではなく、仕事の効率化・スキルアップのような文脈になっていたら1点減点\n- 出したアイデアが5つより多い、少ない場合は1点減点\n- 5つのアイデアのうち、内容が重複しているものがあれば1点減点\n\n\n\n# 言語モデルの回答\n仕事の熱意を取り戻すため